# 所需库

In [1]:
from PyQt5 import QtWidgets,QtCore, QtGui
from PyQt5.QtCore import QSize, Qt
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPicture,QPainter
from PyQt5.QtCore import QPointF,QRectF
import sys
import pyqtgraph as pg
import time
from datetime import datetime
import pandas as pd
import numpy as np

# 模拟数据流

## 全数据

In [2]:
def get_stock_history_from_yahoo(ticker="NFLX"):
    urlstr = "https://finance.yahoo.com/quote/{0}/history?p={0}".format(ticker) 
    hdata = pd.read_html(urlstr)[0][:-1] 
    hdata = hdata.set_index('Date') 
    hdata.index = pd.to_datetime(hdata.index)
    hdata = hdata.astype('d')
    hdata.columns = ['Open','High','Low','Close', 'AdjClose','Volume'] 
    hdata_reindexed=hdata.reset_index()
    valid_set=hdata_reindexed.sort_values(by='Date').reset_index(drop=True)
    return valid_set

In [3]:
def get_stock_history_from_csv(path="nflx.csv"):
    temp=pd.read_csv(path,index_col=0)
    valid_set=temp.sort_values(by='Date').reset_index(drop=True)
    return valid_set

In [4]:
valid_set=get_stock_history_from_csv()

In [5]:
valid_set

,Date,Open,High,Low,Close,AdjClose,Volume
0,2020-02-27,371.46,391.56,370.60,371.71,371.71,10949100.0
1,2020-02-28,364.21,376.77,356.80,369.03,369.03,11178600.0
2,2020-03-02,373.11,381.36,364.50,381.05,381.05,6997900.0
3,2020-03-03,381.03,393.52,367.40,368.77,368.77,8364600.0
4,2020-03-04,377.77,384.01,370.51,383.79,383.79,5487300.0
...,...,...,...,...,...,...,...
95,2020-07-14,517.08,525.50,490.49,524.88,524.88,15083300.0
96,2020-07-15,516.30,529.00,510.18,523.26,523.26,10066700.0
97,2020-07-16,526.48,535.54,504.36,527.39,527.39,24499000.0
98,2020-07-17,494.87,503.59,484.14,492.99,492.99,24991400.0


## 模拟数据流

In [6]:
valid_tuple_list=[]
for i in range(len(valid_set)):
    valid_tuple_list.append(tuple(valid_set.iloc[i]))

In [7]:
valid_tuple_list[1]

('2020-02-28', 364.21, 376.77, 356.8, 369.03, 369.03, 11178600.0)

In [8]:
def get_stock_info():
    global valid_tuple_list
    if len(valid_tuple_list)>0:
        temp=valid_tuple_list[0]
        valid_tuple_list=valid_tuple_list[1:]
        return temp

# 绘制

## 在界面中的图形类

In [9]:
class DrawRecItem(pg.GraphicsObject):
    def __init__(self,data):
        super().__init__()
        self.data=data
        self.draw_rect()
        
    def draw_rect(self):
        
        self.picture=QPicture()
        p1=QPainter(self.picture)
        
        # 设置画pen 颜色，用来画线
        p1.setPen(pg.mkPen((0,0,0)))
        for i in range(len(self.data)):
            #画一条最大值最小值之间的线
            p1.drawLine(QPointF(i,self.data[i][3]),QPointF(i,self.data[i][2]))
            # 设置画刷颜色
            if self.data[i][1]>self.data[i][4]:
                p1.setBrush(pg.mkBrush('g'))
            else:
                p1.setBrush(pg.mkBrush('r'))
            p1.drawRect(QRectF(i-0.3,self.data[i][1],0.6,self.data[i][4]-self.data[i][1]))
         
        
        
    def paint(self,p,*args):
        p.drawPicture(0,0,self.picture)
    def boundingRect(self):
        return QRectF(self.picture.boundingRect())


## 时间轴

In [10]:
class MyAxisItem(pg.AxisItem):
    def __init__(self,ticks,*args,**kwargs):
        pg.AxisItem.__init__(self,*args,**kwargs)
        self.x_values=[x[0] for x in ticks]
        self.x_strings=[x[1] for x in ticks]
    
    def tickStrings(self, values, scale, spacing):
        strings=[]
        for v in values:
            vs=v*scale
            if vs in self.x_values:
                vstr=self.x_strings[np.abs(self.x_values-vs).argmin()]

            else:
                vstr=''
            
            strings.append(vstr)
        return strings

## Update_tab2

In [11]:
#### tab2 线程############################
class Update_tab2(QtCore.QThread):
    requestChange = QtCore.pyqtSignal(tuple)

    def __init__(self, parent=None):
        super(Update_tab2, self).__init__(parent)
        # 设置工作状态与初始num数值

    def __del__(self):
        # 线程状态改变与线程终止
        self.working = False
        self.wait()

    def run(self):
        while True:
            new_data = get_stock_info()
            # 如果一直无信息发送过来，则等待
            if new_data is None:
                #print("waiting for new msg")
                time.sleep(1)
                continue
            else:
                # 通过自定义信号把待显示的字符串传递给槽函数
                self.requestChange.emit(new_data)

## 主界面

In [12]:
## 主基类，是 整个GUI的主窗口，内部含有三个子窗口
class Control_sys_Tab(QTabWidget):
    def __init__(self, parent=None):
        self.RequestRowKey ={}
        self.OrderRowKey  ={}
        self.BatchRowKey  ={}   #To manage batch row index
        self.ErrorRowKey  ={}   #To manage error row index
        self.BuyBatchValue = {}
        self.SellBatchValue= {}
        self.BatchManagers  ={} # To manage value for each batch
        self.g_CurrRequestRow = 0
        self.g_CurrOrderRow  = 0
        self.g_CurrBatchRow  = 0
        self.g_CurrErrorRow  = 0
        super().__init__(parent)

        self.setObjectName("Control_system")
        self.resize(1800, 985)
        self.setWindowTitle("实时监控系统")

        # 创建3个选项卡小控件窗口
        self.tab1 = QWidget()
        self.tab2 = QWidget()
        self.tab3 = QWidget()

        # 将三个选项卡添加到顶层窗口中
        self.addTab(self.tab1, "交易界面")
        self.addTab(self.tab2, "PNL展示界面")
        self.addTab(self.tab3, "PNL定时发送")

        # 记录tab2 中的数据，后续会从其他类中读取数据，并更新和作图
        self.Data=[]
        self.work=Update_tab2()
        

        # 每个选项卡自定义的内容
        self.tab1UI()
        self.tab2UI()
        self.tab3UI()

        self.pushButton.clicked.connect(self.slotStart)
    ################tab1#################################

    def tab1UI(self):
        # 设置主布局
        layout = QHBoxLayout()

        sec_layout = QFormLayout()

        # 创建表格窗口1
        self.tableWidget1 = QtWidgets.QTableWidget()
        self.tableWidget1.setRowCount(5000)
        self.tableWidget1.setColumnCount(11)
        self.tableWidget1.setObjectName("tableWidget")
        self.tableWidget1.setAutoFillBackground(True)
        self.tableWidget1.setHorizontalHeaderLabels(
            ["Acct", "Instrument", "BatchID", "RQID", "Direction", "OrderSize", "TradedVol", "AvgPrice", "Notional",
             "FillRate", "RefPrice"])
        for i in range(0, 11):
            self.tableWidget1.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
        # self.tableWidget.item(0, 0).setFont(font)

        # 表格窗口2
        self.tableWidget2 = QtWidgets.QTableWidget()
        self.tableWidget2.setRowCount(20000)
        self.tableWidget2.setColumnCount(14)
        self.tableWidget2.setObjectName("tableWidget")
        self.tableWidget2.setAutoFillBackground(True)
        self.tableWidget2.setHorizontalHeaderLabels(
            ["OrderRef", "RequestID", "PriceType", "Direction", "OffsetFlag", "HedgeFlag", "LimitPrice", "VolOriginal",
             "VolRemain", "VolTraded", "VolConfirmed", "Status", "OrderSysID", "ExchID"])
        for i in range(0, 14):
            self.tableWidget2.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)

        # 表格窗口3
        self.tableWidget3 = QtWidgets.QTableWidget()
        # self.tableWidget3.setTextAlignment(Qt.AlignHCenter)
        self.tableWidget3.setRowCount(500)
        self.tableWidget3.setColumnCount(6)
        # self.tableWidget3.setStyleSheet('background-repeat:repeat;')  #font color
        # self.tableWidget3.setStyleSheet('color:darkblue;')  #font color
        # self.tableWidget3.setStyleSheet('text-align:center;')
        # self.tableWidget3.setStyleSheet('vertical-align:super;')
        # self.tableWidget3.setStyleSheet('background-color:lightblue')
        # self.tableWidget3.horizontalHeader().setStyleSheet('QHeaderView::section{background:gray}')

        self.tableWidget3.setObjectName("tableWidget")
        self.tableWidget3.setAutoFillBackground(True)
        self.tableWidget3.setHorizontalHeaderLabels(
            ["BatchID", "AcctName", "BuyNotional", "SellNotional", "BuyFillRate", "SellFillRate"])
        for i in range(0, 6):
            self.tableWidget3.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            # self.tableWidget3.
        self.tableWidget3.setColumnWidth(0, 150)
        self.tableWidget3.setColumnWidth(1, 150)
        self.tableWidget3.setColumnWidth(2, 100)
        self.tableWidget3.setColumnWidth(3, 100)
        self.tableWidget3.setColumnWidth(4, 100)
        self.tableWidget3.setColumnWidth(5, 100)
        # self.label = QtWidgets.QLabel(self.centralwidget)
        # self.label.setGeometry(QtCore.QRect(360, 70, 300, 50))
        # self.label.setObjectName("label")
        # self.label.setAutoFillBackground(True)
        # self.label.setAlignment(QtCore.Qt.AlignCenter)
        # self.label.setStyleSheet("border-image:url(images/title.png)")

        self.pushButton = QtWidgets.QPushButton()
        # self.pushButton.setMaximumWidth(100)
        self.pushButton.setObjectName("pushButton")
        self.pushButton.setText("开始运行")

        # 添加表单2进 子布局
        sec_layout.addWidget(self.tableWidget2)

        # 添加表单3进 子布局
        sec_layout.addWidget(self.tableWidget3)

        # 添加 按钮 进 子布局
        sec_layout.addWidget(self.pushButton)

        # 添加表单1 进主布局，子布局进主布局
        layout.addWidget(self.tableWidget1)
        layout.addLayout(sec_layout)

        self.tab1.setLayout(layout)

    #################tab2#################################
    ######################################

    ### 测试线程用代码
    def execute(self):
        # 启动线程
        self.work.start()
        # 线程自定义信号连接的槽函数
        self.work.requestChange.connect(self.display)

    def display(self, new_data):
        # 由于自定义信号时自动传递0个字符串参数，所以在这个槽函数中要接受0个参数
        # print(new_data)
        self.plotData(new_data)



    ########################################
    # 定义一个计时器
#     def timer_start(self):

#         self.timer = QtCore.QTimer(self)
#         self.timer.timeout.connect(self.plotData)
#         self.timer.start(1000)

    def append_stock_data(self):
        temp = get_stock_info()
        if temp != None:
            self.Data.append(temp)

    def plotData(self, new_data):
        # print(len(self.Data))
        if new_data != None:
            item = DrawRecItem(self.Data)

            ## 清空layout2 以重新插入图片
            for i in range(self.layout2.count()):
                self.layout2.itemAt(i).widget().deleteLater()

            index = range(len(self.Data))
            time_list = []
            for i in index:
                temp = self.Data[i][0]
                time_list.append(temp)
            ticks = [(i, j) for i, j in zip(index, time_list)]
            strAxis = MyAxisItem(ticks, orientation="bottom")

            plt = pg.PlotWidget(axisItems={'bottom': strAxis})

            ## 设置背景颜色
            plt.setBackground((255, 255, 255))
            
            # 将iTem加入到plotwidget控件中
            plt.addItem(item)

            # 将控件添加到pyqt中
            self.layout2.addWidget(plt)
            # 将layout 布局添加到 tab2中
            self.tab2.setLayout(self.layout2)
            self.Data.append(new_data)

    def tab2UI(self):
        #         self.timer_start()
        self.layout2 = QVBoxLayout()
        #self.timer_start()
        self.execute()

    def tab3UI(self):

        return

    ####################################################################################################################
    ##############################################################
    #     PyQt5 中的pyQtslot 是python中的decorator，用其可以将一个method 定义为 槽

    #     槽的传参方式 主要是直接传入一个 函数指针

    ##############################################################

    @QtCore.pyqtSlot()
    def slotStart(self):
        # 按钮 暂停使用
        self.pushButton.setEnabled(False)
        # 开启一个新进程用来 更新数据
        self.update_data_thread = UpdateData(self)
        self.update_data_thread.requestChanged.connect(self.onRequestChanged)
        # 线程进入 准备阶段
        self.update_data_thread.start()

    @QtCore.pyqtSlot(int, int, str)
    def onRequestChanged(self, row, msgType, text):
        elems = text.split(',')
        # row =0
        column = 0
        if (msgType == 1):
            myKey = self.RequestRowKey.get(elems[2] + elems[3])
            if (myKey == None):
                myKey = self.g_CurrRequestRow
                self.g_CurrRequestRow += 1
                self.RequestRowKey[elems[2] + elems[3]] = myKey  ### CREATE A REQUEST ROW

            for ele in elems:
                it = self.tableWidget1.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget1.setItem(myKey, column, it)
                if (column == 0):
                    it.setText(AcctNameDict[int(ele)])
                    # print("What is this: %d, and acctName =  %s"%(int(ele),AcctNameDict[int(ele)]))
                elif (column == 4):
                    it.setText(directionType[ele])
                else:
                    it.setText(ele)
                column += 1
            self.tableWidget1.selectRow(myKey)

            myKey = self.BatchRowKey.get(elems[2])
            if (myKey == None):  ### UPDATE ACCT-BATCH TABLE HERE
                myKey = self.g_CurrBatchRow
                self.g_CurrBatchRow += 1
                self.BatchRowKey[elems[2]] = myKey
                self.BatchManagers[elems[2]] = BatchManager()  ### CREATE A BATCH MANAGER
                self.BatchManagers[elems[2]].bookAcctID(int(elems[0]))
            # print("what is the direction code: %s"%(elems[4]))
            myTradeDirection = 1.0 if (int(elems[4]) == 0) else -1.0
            self.BatchManagers[elems[2]].bookTotalValue(elems[3], float(elems[5]) * float(elems[10]) * myTradeDirection)
            self.BatchManagers[elems[2]].bookTradedValue(elems[3], float(elems[8]))  ### need to change here
            print(elems[3], float(elems[8]))

            ### FILL INFORMATION IN THIS ROW
            ###print("GET NOTIONA:",str(self.BatchManagers[elems[2]].getBuyNotional()),str(self.BatchManagers[elems[2]].getSellNotional()))
            thisAcctID = self.BatchManagers[elems[2]].getAcctID()
            thisAcctIDStr = AcctNameDict[thisAcctID]
            vals1 = self.BatchManagers[elems[2]].getBuyNotional()
            vals2 = self.BatchManagers[elems[2]].getSellNotional()
            vals = [elems[2], thisAcctIDStr, "{:,.2f}".format(vals1[0]), "{:,.2f}".format(vals2[0]),
                    "{:.2%}".format(vals1[1]), "{:.2%}".format(vals2[1])]
            for column in range(0, 6):
                it = self.tableWidget3.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget3.setItem(myKey, column, it)
                it.setText(vals[column])
            self.tableWidget3.selectRow(myKey)

        elif (msgType == 2):
            myKey = self.OrderRowKey.get(elems[0] + elems[1] + elems[12])
            if (myKey == None):
                myKey = self.g_CurrOrderRow
                self.g_CurrOrderRow += 1
                self.OrderRowKey[elems[0] + elems[1] + elems[12]] = myKey
            for ele in elems:
                it = self.tableWidget2.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget2.setItem(myKey, column, it)  # can be done in dictionary of dictionary way, better
                if (column == 3):
                    it.setText(directionType[ele])
                elif (column == 4):
                    it.setText(OpenCloseFlag[ele])
                elif (column == 5):
                    it.setText(hedgeFlag[ele])
                elif (column == 11):
                    it.setText(orderStatus[ele])
                else:
                    it.setText(ele)
                column += 1
            self.tableWidget2.selectRow(myKey)

        elif (msgType == 3):
            myKey = self.ErrorRowKey.get(elems[0] + elems[2])
            if (myKey == None):
                myKey = self.g_CurrErrorRow
                self.g_CurrErrorRow += 1
                self.ErrorRowKey[elems[0] + elems[2]] = myKey
            for ele in elems:
                it = self.tableWidget4.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget4.setItem(myKey, column, it)  # can be done in dictionary of dictionary way, better
                it.setText(ele)
                column += 1
            self.tableWidget4.selectRow(myKey)
        else:
            self.tableWidget1.selectRow(row)

# Main

In [13]:
if __name__ == "__main__":
    app = QtWidgets.QApplication.instance()
    if app is None:
         app = QtWidgets.QApplication(sys.argv)
    w = Control_sys_Tab()
    w.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\taosu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
